## Experimental Result

In [1]:
from z3 import *
import time

### Synthesize Switched Controller

In [2]:
t = Real('t')

def StateCal(flows, pred1, pred2, formula_simp = 1):
    tau = Real('tau')
    pred2 = And(pred1, pred2)
    for var in flows.keys():
        pred2 = substitute(pred2, (var, var + flows[var]*tau))
        #pred2 = substitute(pred2, (var, substitute(flows[var], (t,tau))))
        #print(pred2)
    pred2 = substitute(pred2, (t, t + tau))
    #print(Exists(tau, And(tau>0,pred2)))
    tac = Tactic('qe2')
    #print('qe start') 
    qe_resul = tac(Exists(tau, And(tau >= 0, pred2))).as_expr()
    #print('qe finished')
    if (formula_simp == 1):
        return Tactic('unit-subsume-simplify').apply(And(qe_resul, pred1, t >= 0)).as_expr() # Default, return the simplified formula
    elif (formula_simp == 0):
        return And(qe_resul, pred1, t >= 0) # Use this return if do not need simplify. Simplify is potentially expensive
 
# Check the correctness of the StateCal()
#s = Solver()
#fun_resul = StateCal({x:1},And(x>=0, x<= 4),p2)
#dir_resul = And(tac(Exists(tau, p2t)).as_expr(),
#                And(x >= 0, x <= 4, t>=0))
#s.add(Or(And(fun_resul, 
#             Not(dir_resul)), 
#         And(Not(fun_resul), 
#             dir_resul)))
#if(s.check()==unsat):
#    print("equivalent")
#    print(fun_resul)
#else: 
#    print("unequivalent")

def repl(formula,var,val):
     assign_val = Then(Tactic('qe2'),Tactic('ctx-solver-simplify'),Tactic('unit-subsume-simplify'))#,Tactic('ctx-solver-simplify'))
     #assign_val = Tactic('qe2')
     return assign_val(Exists(var, And(var == val, formula))).as_expr()

def TimeRelevantCal(x, E, flows, phi1, phi2, k, prt = 0):
     n = len(x) # number of variables
     m = len(E) # number of modes

     X = [[BoolVal(False) for switch_time in range(k)] for q in range(m)] # time-relevant predicate - X[q][switch_time]
     Init = [[BoolVal(False) for switch_time in range(k)] for q in range(m) ] # initial condition include at leat i times switch - Init[q][switch_time]
     Jump = [[[BoolVal(False) for switch_time in range(k)] for post_q in range(m)] for q in range(m)] # time-relevant switch condition corresponding to init - Jump[q][post_q][switch_time]


     for q in range(m):
          q_flow = dict(zip(x,flows[q]))
          X[q][0] = StateCal(q_flow, phi1, phi2)
          Init[q][0] = Tactic('simplify').apply(And(repl(X[q][0],t,0), t == 0)).as_expr()
          if (prt):
               print(f"=========================X_{q}^0=========================")
               print(X[q][0])

     i = 1
     while(i < k):
          for q in range(m):
               q_flow = dict(zip(x,flows[q]))
               for post_q in range(m):
                    if (E[q][post_q]):
#                         print(f"phi2 : X_{post_q}^{i-1}={X[post_q][i-1]}")
                         #X[q][i] = Tactic('ctx-solver-simplify').apply(Or(X[q][i],StateCal(q_flow, phi1, X[post_q][i-1]))).as_expr()
                         #X[q][i] = Tactic('unit-subsume-simplify').apply(Or(X[q][i],StateCal(q_flow, phi1, X[post_q][i-1]))).as_expr()
                         X[q][i] = Or(X[q][i],StateCal(q_flow, phi1, X[post_q][i-1]))
                         Jump[q][post_q][i] = X[post_q][i-1]
                         #Jump[q][post_q][i] = Tactic('ctx-solver-simplify').apply(X[post_q][i-1]).as_expr()
               Init[q][i] = Tactic('simplify').apply(And(repl(And(X[q][i],Not(X[q][i-1])), t, 0), t == 0)).as_expr()
               #print(Init[q][i])
               if (prt):
                    print(f"=========================X_{q}^{i}=========================")
                    print(f"{X[q][i]}")
               print(f"X[{q}][{i}] finished")
          i = i + 1 
     
     return X, Init, Jump

### Synthesize Switched Hybrid Automata

In [4]:
def simplifyExpr(guard):
    simplified = simplify(guard)
    tac = Then("ctx-solver-simplify", "propagate-ineqs",
                  Repeat(OrElse(Tactic('split-clause'), Tactic('skip'))),
                  "ctx-solver-simplify", "propagate-ineqs", "tseitin-cnf")
    simplified_guard = tac(simplified).as_expr()
    return simplify(simplified_guard)

def Reach(flows, init, formula_simp = 1):
    tau = Real('tau')
    #print(flows.keys())
    for var in flows.keys():
        #print(init)
        init = substitute(init, (var, var - flows[var]*tau))
    init = substitute(init, (t, t - tau))
    #print(init)
    tac = Tactic('qe2')

    reach_set = tac(Exists(tau, And(tau >= 0, init))).as_expr()
    if (formula_simp == 1):
        return Tactic('unit-subsume-simplify').apply(reach_set).as_expr() # Default, return the simplified formula
    elif (formula_simp == 0):
        return reach_set # Use this return if do not need simplify. Simplify is potentially expensive

def JumpCal(flows, init, jump_t):
    x = list(flows.keys())
    n = len(x)

    lb = [Real('lb%s' % i) for i in range(n)]
    ub = [Real('ub%s' % i) for i in range(n)]
    jump = BoolVal(True)
    for i in range(n):
        jump = And(jump, lb[i] <= x[i], x[i] <= ub[i])
    #print(jump)    
    reach_set = Reach(flows, init)
    #print(reach_set)
    tac = Tactic('qe2')

    qe_resul = tac(ForAll(x+[t], Implies(And(reach_set, jump), jump_t))).as_expr()
    #print(qe_resul)
    qe_resul = tac(Exists(x+[t], And(qe_resul, jump, reach_set))).as_expr()
    #print(qe_resul)
    for i in range(n):
        qe_resul = And(qe_resul, lb[i] <= ub[i])
    #print(qe_resul)
    #jump_cond = simplifyExpr(qe_resul)
    jump_cond = simplify(qe_resul)
    #print(jump_cond)
    return jump_cond

def NonEmptyCheck(pred):
    s = Solver()
    s.add(pred)
    if(s.check() == sat):
        return 1
    else:
        return 0

def ConflictCheck(pred):
    s = Solver()
    s.add(pred)
    if(s.check() == unsat):
        return 1
    else:
        return 0
    
def GuardSyn(x, boundary):
    n = len(x)
    lb = [Real('lb%s' % i) for i in range(n)]
    ub = [Real('ub%s' % i) for i in range(n)]

    s = Solver()
    s.add(boundary)
    if(s.check() == sat):
        boundary_val = s.model()
        if(len(boundary_val) == 0):
            return BoolVal(False)
        else:
            jump = BoolVal(True)
            for i in range(len(x)):
                jump = And(jump, boundary_val[lb[i]] <= x[i], x[i] <= boundary_val[ub[i]])
            return jump
    else:
        return BoolVal(False)

def SwitchSyn(x, E, flows, given_inits, jumps, inits, k):
    m = len(E)
    n = len(x)


    for q in range(m):
        for i in range(k):
            #inits[q][i] = Tactic('unit-subsume-simplify').apply(And(inits[q][i], given_inits[q])).as_expr()
            inits[q][i] = And(inits[q][i], given_inits[q])
        inits[q][1] = Or(inits[q][0], inits[q][1])
    
    #print(inits)
    
    jump_cond = [[BoolVal(True) for post_q in range(m)] for q in range(m)]
    Jumps =  [[BoolVal(False) for post_q in range(m)] for q in range(m)]
    for q in range(m):
        for post_q in range(m):
            if E[q][post_q] == 1:
                flow = dict(zip(x, flows[q]))
                #print(flow)
                for i in range(1,k):
                    if (NonEmptyCheck(inits[q][i])):
                        #print(f"Jump condition from Init[q{q}][{i}] to Jump[q{q}][q{post_q}][{i}]:",JumpCal(flow,inits[q][i], jumps[q][post_q][i]))
                        #print(f"start calculate jump conditions for Jump[{q}][{post_q}][{i}]")
                        temp_jump = JumpCal(flow, inits[q][i], jumps[q][post_q][i])
                        #print("Calculation finished")
                        if(ConflictCheck(And(jump_cond[q][post_q], temp_jump)) == 0):
                            print(f"No conflicts when adding {i}-times jump condition from q{q} to q{post_q}")
                            jump_cond[q][post_q] = And(jump_cond[q][post_q], temp_jump)
                        elif(ConflictCheck(And(jump_cond[q][post_q], temp_jump)) and i < k-1):
                            print(f"Find a conflict when adding Jump[{q}][{post_q}][{i}] to jump condition")
                            inits[q][i+1] = Or(inits[q][i+1], inits[q][i])
                            #inits[q][i] = BoolVal(False)
                            print("Solve it by enlarge time-relevant jump condition")
                        elif(ConflictCheck(And(jump_cond[q][post_q], temp_jump)) and i == k):
                            #inits[q][i] = BoolVal(False)
                            print("Enlarge fail, eliminate the initial set")
                    #print(f"condition of lb,ub for jump[{q}][{post_q}]: {simplifyExpr(jump_cond[q][post_q])}")
                    #s = Solver()
                    #s.add(jump_cond[q][post_q])
                    #if(s.check() == unsat):
                    #if(ConflictCheck(jump_cond[q][post_q])):
                        # shrink the initial set/ enlarge the jump condition
                        # print(f"Find a conflict when adding Jump[{q}][{post_q}][{i}] to jump condition")
                        # break
                #if(ConflictCheck(jump_cond[q][post_q]) == 0):
                #print(f"constrain of jump condition from q{q} to q{post_q}: {jump_cond[q][post_q]}")
                Jumps[q][post_q] = GuardSyn(x,jump_cond[q][post_q])
                # Add the initial condition to init[post_q][i-1]
                for i in range(2,k):
                    inits[post_q][i-1] = Or(inits[post_q][i-1], And(Jumps[q][post_q], Reach(flow, inits[q][i])))
                    #print(f"inits[{q}][{i}] = {inits[q][i]}")
                    #print(f"reachable set from inits[{q}][{i}] = {Reach(flow, inits[q][i])}")
                #print(inits[post_q])
    for q in range(m):
        for post_q in range(m):
            if(E[q][post_q]):
                print(f"Jumps[{q}][{post_q}] = {Jumps[q][post_q]}")
    
                                    

            


    

### Benchmarks

#### Water Tank System - Running Example

In [6]:
n = 1 # number of variables
m = 2 # number of modes
k = 5 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0,1],
     [1,0]] # structure of control graph, represented by adjacency matrix
flows = [[1],
        [-1]] # flow condition for each mode
#flows = [[x[0] + t],
#         [ x[0] - t]] # flow condition for each mode

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= 0, x[0] <= 4) 
phi2 = And(x[0] >= 3, x[0] <= 5, t <= 4, t >= 3)
#phi2 = And(x[0] >= 3, t>= 3, t <= 4)

given_Init = [And(0 <= x[0], x[0] <= 3), And(3 <= x[0], x[0] <= 5)]

start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

start2 = time.perf_counter()
#SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s    {end2-start2}s")

prt = 0
chk = 1
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={simplifyExpr(Init[0][i])}")
          print(f"Init[1][{i}]={simplifyExpr(Init[1][i])}")
     print("================================================================")
     for q in range(2):
          for post_q in range(2):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")
if(chk):
     for q in range(m):
          for i in range(k-1):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")



X[0][1] finished
X[1][1] finished
X[0][2] finished
X[1][2] finished
X[0][3] finished
X[1][3] finished
X[0][4] finished
X[1][4] finished
                     Alg1                 Alg2      
Total Time  : 0.06938595895189792s    9.833020158112049e-06s
X[0][0] is subset of X[0][1], result correct
X[0][1] is subset of X[0][2], result correct
X[0][2] = X[0][3]
X[0][3] = X[0][4]
X[1][0] is subset of X[1][1], result correct
X[1][1] = X[1][2]
X[1][2] = X[1][3]
X[1][3] = X[1][4]


#### Nuclear Reactor


##### Simple System with 4 modes 4 edge


- Control Modes:
    - q0 : inc-cooling, $\dot x_0 = 3, \dot x_1 = 1$; 
    - q1 : hig-cooling, $\dot x_0 = 0.5, \dot x_1 = 0$; 
    - q2 : dec-cooling, $\dot x_0 = -3, \dot x_1 = -1$; 
    - q3 : low-cooling, $\dot x_0 = -0.5, \dot x_1 = 0$;
- Control Switch: 
    - low-cooling -> inc-cooling, 
    - inc-cooling -> hig-cooling,
    - hig-cooling -> dec-cooling, 
    - dec-cooling -> low-cooling.

- Initial Conditions:

    \begin{align*}
        &\texttt{Init}(low-cooling) = (30 \leq x_1 \leq 50)\wedge (x_2 == 0),\\
        &\texttt{Init}(others) = False
    \end{align*}

- St-RA formula:

    \begin{align*}
        \varphi = \left((0\le x_1\le 1)\wedge(10\le x_0 \le 90)\right)\mathcal{U}_{[5,7]}\left(60\le x_0\le 75\right)
    \end{align*}

In [12]:
n = 2 # number of variables
m = 4 # number of modes
k = 7 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0,1,0,0],
     [0,0,1,0],
     [0,0,0,1],
     [1,0,0,0]] # structure of control graph, represented by adjacency matrix

#flows = [[x[0] - t, x[1] + t],
#         [x[0] - 2*t, x[1]],
#         [x[0] - t, x[1] - t],
#         [x[0] + 4*t, x[1]]] # flow condition for each mode
flows = [[-1,  1],
        [-2,  0],
        [-1, -1],
        [ 4,  0]]

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= 10, x[0] <= 90, x[1] >= 0, x[1] <= 1) 
phi2 = And(x[0] >= 40, x[0] <= 50, t>= 15, t <= 20)

given_Init = [And(x[1] == 0, x[0]>= 20, x[0] <=60), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)
print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s    {end2-start1}s")

prt = 0
chk = 1
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={simplifyExpr(Init[0][i])}")
          print(f"Init[1][{i}]={simplifyExpr(Init[1][i])}")
          print(f"Init[2][{i}]={simplifyExpr(Init[2][i])}")
          print(f"Init[3][{i}]={simplifyExpr(Init[3][i])}")
          print("================================================================")
     #for q in range(4):
     #     for post_q in range(4):
     #          for i in range(k):
     #               print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
     #          print("================================================================")


if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")




======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
======================Calculating Time-Irrelevant Jump Conditions======================
No conflicts when adding 3-times jump condition from q0 to q1
No conflicts when adding 2-times jump condition from q1 to q2
No conflicts when adding 1-times jump condition from q2 to q3
Jumps[0][1] = And(And(True, 0 <= x0, 20 >= x0), 0 <= x1, 0 >= x1)
Jumps[1][2] = And(And(True, 10 <= x0, 10 >= x0), 0 <= x1, 0 >= x1)
Jumps[2][3] = And(And(True, 10 <= x0, 10 >= x0), -1 <= x1, 0 >= x1)
Jumps[3][0] = False
           

##### Complex System with 8 modes

In [13]:

n = 2 # number of variables
m = 8 # number of modes
k = 7 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0,1,1,1,0,0,0,0],
     [1,0,1,1,0,0,0,0],
     [1,1,0,1,0,0,0,0],
     [0,0,0,0,1,1,1,0],
     [0,0,0,0,0,1,1,1],
     [0,0,0,0,1,0,1,1],
     [0,0,0,0,1,1,0,1],
     [1,1,1,0,0,0,0,0]] # structure of control graph, represented by adjacency matrix

#flows = [[x[0] - 2*t, x[1] + t],
#         [x[0] - 1*t, x[1] + 0.7*t],
#         [x[0] - 0.5*t, x[1] + 0.2*t],
#         [x[0] - 3*t, x[1]],
#         [x[0] - 2*t, x[1] - t],
#         [x[0] - 1*t, x[1] - 0.7*t],
#         [x[0] - 0.5*t, x[1] - 0.2*t],
#         [x[0] + 4*t, x[1]]] # flow condition for each mode
flows = [[-2  ,  1  ],
         [-1  ,  0.7],
         [-0.5,  0.2],
         [-3  ,  0  ],
         [-2  , -1  ],
         [-1  , -0.7],
         [-0.5, -0.2],
         [ 4  ,  0  ]] # flow condition for each mode

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= 10, x[0] <= 90, x[1] >= 0, x[1] <= 1) 
phi2 = And(x[0] >= 65, x[0] <= 69, t>= 15, t <= 20)

given_Init = [And(x[1] == 0, x[0]>= 50, x[0] <=90), BoolVal(False), BoolVal(False), BoolVal(False), 
              BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)
print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")


prt = 0
chk = 1
if(prt):
     for i in range(k):
          for q in range(m):
               s = Solver()
               s.add(Init[q][i])
               if (s.check() == sat):
                    print(f"Init[{q}][{i}] is non-empty")
               else:
                    print(f"Init[{q}][{i}] is empty")
          print("================================================================")
     #for q in range(4):
     #     for post_q in range(4):
     #          for i in range(k):
     #               print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
     #          print("================================================================")


if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")

======================Calculating Time-Relevant Jump Conditions======================


X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[7][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[7][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[7][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[4][4] finished
X[5][4] finished
X[6][4] finished
X[7][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[4][5] finished
X[5][5] finished
X[6][5] finished
X[7][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[4][6] finished
X[5][6] finished
X[6][6] finished
X[7][6] finished
======================Calculating Time-Irrelevant Jump Conditions======================
Find a conflict when adding Jump[0][1][3] to jump condition
Solve it by enlarge time-relevant ju

##### Complex system with 10 modes

In [14]:

n = 2 # number of variables
m = 10 # number of modes
k = 7 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0,1,1,1,1,0,0,0,0,0],
     [1,0,1,1,1,0,0,0,0,0],
     [1,1,0,1,1,0,0,0,0,0],
     [1,1,1,0,1,0,0,0,0,0],
     [0,0,0,0,0,1,1,1,1,0],
     [0,0,0,0,0,0,1,1,1,1],
     [0,0,0,0,0,1,0,1,1,1],
     [0,0,0,0,0,1,1,0,1,1],
     [0,0,0,0,0,1,1,1,0,1],
     [1,1,1,1,0,0,0,0,0,0]] # structure of control graph, represented by adjacency matrix

#flows = [[x[0] - 2*t, x[1] + t],
#         [x[0] - 1*t, x[1] + 0.7*t],
#         [x[0] - 0.5*t, x[1] + 0.2*t],
#         [x[0] - 3*t, x[1]],
#         [x[0] - 2*t, x[1] - t],
#         [x[0] - 1*t, x[1] - 0.7*t],
#         [x[0] - 0.5*t, x[1] - 0.2*t],
#         [x[0] + 4*t, x[1]]] # flow condition for each mode
flows = [[-2  ,  1  ],
         [-1  ,  0.7],
         [-0.8,  0.4],
         [-0.5,  0.2],
         [-3  ,  0  ],
         [-2  , -1  ],
         [-1  , -0.7],
         [-0.8, -0.4],
         [-0.5, -0.2],
         [ 4  ,  0  ]] # flow condition for each mode

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= 10, x[0] <= 90, x[1] >= 0, x[1] <= 1) 
phi2 = And(x[0] >= 65, x[0] <= 69, t>= 15, t <= 20)

given_Init = [And(x[1] == 0, x[0]>= 50, x[0] <=90), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), 
              BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)
print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")


prt = 0
chk = 1
if(prt):
     for i in range(k):
          for q in range(m):
               s = Solver()
               s.add(Init[q][i])
               if (s.check() == sat):
                    print(f"Init[{q}][{i}] is non-empty")
               else:
                    print(f"Init[{q}][{i}] is empty")
          print("================================================================")
     #for q in range(4):
     #     for post_q in range(4):
     #          for i in range(k):
     #               print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
     #          print("================================================================")


if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")

======================Calculating Time-Relevant Jump Conditions======================


X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[7][1] finished
X[8][1] finished
X[9][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[7][2] finished
X[8][2] finished
X[9][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[7][3] finished
X[8][3] finished
X[9][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[4][4] finished
X[5][4] finished
X[6][4] finished
X[7][4] finished
X[8][4] finished
X[9][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[4][5] finished
X[5][5] finished
X[6][5] finished
X[7][5] finished
X[8][5] finished
X[9][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[4][6] finished
X[5][6] finished
X[6][6] finished
X[7][6] finished
X[8][6] finish

#### Car-Sequence System

##### 4-car-sequence

In [15]:
n = 4 # number of variables
m = 2**n # number of modes
k = 4 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0]]

flows = []
coef = [[7,1],[5,3],[6,1],[5,4]] # the coefficient of ODE for each car - coef[car][mode:hight-speed, low-speed]
for q in range(m):
     temp_q = q
     temp_flow = []
     for i in range(n):
          if(temp_q % 2 == 0):
               temp_flow.append(coef[i][0])
          if(temp_q % 2 == 1):
               temp_flow.append(coef[i][1])
          temp_q = temp_q//2
     flows.append(temp_flow) # write flow conditions in each mode as a 2-dim list
#flows = [flows[0], flows[1], flows[4], flows[5], flows[6], flows[7], flows[8], flows[9], flows[12], flows[13], flows[14], flows[15]]

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= x[1] + 1, x[0] <= x[1] +3, x[1] >= x[2] + 1, x[1] <= x[2] + 3, x[2] >= x[3] + 1, x[2] <= x[3] + 3) 
phi2 = And(x[0] >= 20, x[0] <= 45, t>= 5, t <= 8)

given_Init = [And(x[0] >= x[1] + 1, x[1] >= x[2] + 1, x[2] >= x[3] + 1, 0 <= x[0], x[0] <= 20, 0 <= x[1], x[1] <= 20, 0 <= x[2], x[2] <= 20, 0 <= x[3], x[3] <= 20), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)


print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
#SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")


======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[7][1] finished
X[8][1] finished
X[9][1] finished
X[10][1] finished
X[11][1] finished
X[12][1] finished
X[13][1] finished
X[14][1] finished
X[15][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[7][2] finished
X[8][2] finished
X[9][2] finished
X[10][2] finished
X[11][2] finished
X[12][2] finished
X[13][2] finished
X[14][2] finished
X[15][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[7][3] finished
X[8][3] finished
X[9][3] finished
X[10][3] finished
X[11][3] finished
X[12][3] finished
X[13][3] finished
X[14][3] finished
X[15][3] finished
======================Calculating Time-Irrelevant Jump Conditions===============

##### 3-car-sequence

In [ ]:


n = 3 # number of variables
m = 2**n # number of modes
k = 9 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables


E = [[0, 1, 1, 0, 1, 0, 0, 0],
 [1, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 0, 1, 0, 0, 1, 0],
 [0, 1, 1, 0, 0, 0, 0, 1],
 [1, 0, 0, 0, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 0, 1, 0, 1, 1, 0]]


flows = []
coef = [[7,1],[5,3],[6,1]] # the coefficient of ODE for each car - coef[car][mode:hight-speed, low-speed]
for q in range(m):
     temp_q = q
     temp_flow = []
     for i in range(n):
          if(temp_q % 2 == 0):
               temp_flow.append(coef[i][0])
          if(temp_q % 2 == 1):
               temp_flow.append(coef[i][1])
          temp_q = temp_q//2
     flows.append(temp_flow) # write flow conditions in each mode as a 2-dim list
#flows = [flows[0], flows[1], flows[4], flows[5], flows[6], flows[7], flows[8], flows[9], flows[12], flows[13], flows[14], flows[15]]

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
#phi1 = And(x[0] >= x[1] + 1, x[0] <= x[1] +5, x[1] >= x[2] + 1, x[1] <= x[2] + 5) 
#phi2 = And(x[0] >= 20, x[0] <= 25, t>= 2, t <= 3)
     
#================ k=9 fix-point =============================
phi1 = And(x[0] >= x[1]+1, x[0] <= x[1] + 3, x[1] >= x[2]+1) 
phi2 = And(x[0] >= 20, x[0] <= 25, t>= 2, t <= 3)


given_Init = [And(x[0] >= x[1] + 1, x[1] >= x[2] + 1, 0 <= x[0], x[0] <= 20, 0 <= x[1], x[1] <= 20, 0 <= x[2], x[2] <= 20), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)
print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k,0)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
#SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")


======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[7][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[7][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[7][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[4][4] finished
X[5][4] finished
X[6][4] finished
X[7][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[4][5] finished
X[5][5] finished
X[6][5] finished
X[7][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[4][6] finished
X[5][6] finished
X[6][6] finished
X[7][6] finished
X[0][7] finished
X[1][7] finished
X[2][7] finished
X[3][7] finished
X[4][7] finished
X[5][7] finis

In [17]:

n = 2 # number of variables
m = 2**n # number of modes
k = 9 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0, 1, 0, 1],
     [1, 0, 1, 0],
     [0, 1, 0, 1],
     [1, 0, 1, 0]]

flows = []
coef = [[7,4],[6,5]] # the coefficient of ODE for each car - coef[car][mode:hight-speed, low-speed]
for q in range(m):
     temp_q = q
     temp_flow = []
     for i in range(n):
          if(temp_q % 2 == 0):
               temp_flow.append(coef[i][0])
          if(temp_q % 2 == 1):
               temp_flow.append(coef[i][1])
          temp_q = temp_q//2
     flows.append(temp_flow) # write flow conditions in each mode as a 2-dim list
#flows = [flows[0], flows[1], flows[4], flows[5], flows[6], flows[7], flows[8], flows[9], flows[12], flows[13], flows[14], flows[15]]

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= x[1] + 1, x[0] <= x[1] +3) 
phi2 = And(x[0] >= 20, x[0] <= 45, t>= 5, t <= 8)

given_Init = [And(x[1] + 1 <= x[0], 0 <= x[0], x[0] <= 40, 0 <= x[1], x[1] <= 40), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)

print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")



======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[0][7] finished
X[1][7] finished
X[2][7] finished
X[3][7] finished
X[0][8] finished
X[1][8] finished
X[2][8] finished
X[3][8] finished
======================Calculating Time-Irrelevant Jump Conditions======================
Find a conflict when adding Jump[0][1][2] to jump condition
Solve it by enlarge time-relevant jump condition
No conflicts when adding 3-times jump condition from q0 to q1
Find a conflict when adding Jump[0][1][5] to jump condition
Solve it by enlarge time-relevant jump condition
Fi

##### 2-car-sequence

In [ ]:

n = 2 # number of variables
m = 2**n # number of modes
k = 9 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0, 1, 0, 1],
     [1, 0, 1, 0],
     [0, 1, 0, 1],
     [1, 0, 1, 0]]

flows = []
coef = [[7,4],[6,5]] # the coefficient of ODE for each car - coef[car][mode:hight-speed, low-speed]
for q in range(m):
     temp_q = q
     temp_flow = []
     for i in range(n):
          if(temp_q % 2 == 0):
               temp_flow.append(coef[i][0])
          if(temp_q % 2 == 1):
               temp_flow.append(coef[i][1])
          temp_q = temp_q//2
     flows.append(temp_flow) # write flow conditions in each mode as a 2-dim list
#flows = [flows[0], flows[1], flows[4], flows[5], flows[6], flows[7], flows[8], flows[9], flows[12], flows[13], flows[14], flows[15]]

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] >= x[1] + 1, x[0] <= x[1] +3) 
phi2 = And(x[0] >= 20, x[0] <= 25, t>= 2, t <= 3)

given_Init = [And(x[1] + 1 <= x[0], 0 <= x[0], x[0] <= 40, 0 <= x[1], x[1] <= 40), BoolVal(False), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)

print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")



======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[0][7] finished
X[1][7] finished
X[2][7] finished
X[3][7] finished
X[0][8] finished
X[1][8] finished
X[2][8] finished
X[3][8] finished
======================Calculating Time-Irrelevant Jump Conditions======================
Find a conflict when adding Jump[0][1][1] to jump condition
Solve it by enlarge time-relevant jump condition
No conflicts when adding 2-times jump condition from q0 to q1
Find a conflict when adding Jump[0][1][3] to jump condition
Solve it by enlarge time-relevant jump condition
Fi

#### Water Tanks

##### 2 Tank

Consider battery package consisting of 2 batteries. 

\begin{align*}
& x_0 \text{ - water level of tank 0}\\
& x_1 \text{ - water level of tank 1}
\end{align*}

- Control Mode: c - charge; d - discharge
    - q0 - c01d01: $\dot x_0 = 0,5, \dot x_1 = -0.5$
    - q1 - c01u0 : $\dot x_0 = -0.5, \dot x_1 = 0.8$
    - q2 - c01u1 : $\dot x_0 = 1.4, \dot x_1 = -1.5$
    - q3 - c0u01 : $\dot x_0 = 3, \dot x_3 = -1.5$
    - q4 - c0u0  : $\dot x_0 = 2, \dot x_1 = -0.2$
    - q5 - c1u01 : $\dot x_0 = -1, \dot x_1 = 1.5$
    - q6 - c1u1  : $\dot x_0 = -0.1, \dot x_1 = 0.5$
- Control Switch
    - q0 --> q1, q0 --> q5,
    - q1 --> q2, q1 --> q4,
    - q2 --> q1, q2 --> q6,
    - q3 --> q4, q3 --> q5,
    - q4 --> q1,
    - q5 --> q0, q5 --> q3
    - q6 --> q2

##### $\varphi_1$

In [18]:

n = 2 # number of variables
m = 7 # number of modes
k = 10 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0 for i in range(m)] for j in range(m)]

E[0][1] = 1; E[0][5] = 1
E[1][2] = 1; E[1][4] = 1
E[2][1] = 1; E[2][6] = 1
E[3][4] = 1; E[3][5] = 1
E[4][1] = 1
E[5][0] = 1; E[5][3] = 1
E[6][2] = 1

#flows = [[x[0] + 0.5*t, x[1] - 0.5*t],
#         [x[0] - 0.5*t, x[1] + 0.8*t],
#         [x[0] + 1.4*t, x[1] - 1.5*t],
#         [x[0] + 3*t,   x[1] - 1.5*t],
#         [x[0] + 2*t,   x[1] - 0.2*t],
#         [x[0] - 1*t,   x[1] + 1.5*t],
#         [x[0] - 0.1*t, x[1] + 0.5*t]]
flows = [[ 0.5, -0.5],
         [-0.5,  0.8],
         [ 1.4, -1.5],
         [ 3  , -1.5],
         [ 2  , -0.2],
         [-1  ,  1.5],
         [-0.1,  0.5]]

# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4) 
phi1 = And(x[0] - x[1] <= 10, x[0] - x[1] >= -10, x[0] >= 10, x[0] <= 95, x[1] >= 10, x[1] <= 95) 
phi2 = And(x[0] >= 50, x[0] <= 80, x[1] >= 50, x[1] <= 80, t>= 50, t <= 60)

given_Init = [And(40 <= x[0], x[0] <= 60, 40 <= x[1], x[1] <= 60, x[1] - x[0] <= 10), BoolVal(False), BoolVal(False), BoolVal(False), And(40 <= x[0], x[0] <= 60, 40 <= x[1], x[1] <= 60, x[1] - x[0] <= 10), BoolVal(False), And(40 <= x[0], x[0] <= 60, 40 <= x[1], x[1] <= 60, x[1] - x[0] <= 10)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)

print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")



======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[4][4] finished
X[5][4] finished
X[6][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[4][5] finished
X[5][5] finished
X[6][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[4][6] finished
X[5][6] finished
X[6][6] finished
X[0][7] finished
X[1][7] finished
X[2][7] finished
X[3][7] finished
X[4][7] finished
X[5][7] finished
X[6][7] finished
X[0][8] finished
X[1][8] finished
X[2][8] finished
X[3][8] finished
X[4][8] finis

##### $\varphi_2$

In [19]:

n = 2 # number of variables
m = 7 # number of modes
k = 8 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0 for i in range(m)] for j in range(m)]

E[0][1] = 1; E[0][5] = 1
E[1][2] = 1; E[1][4] = 1
E[2][1] = 1; E[2][6] = 1
E[3][4] = 1; E[3][5] = 1
E[4][1] = 1
E[5][0] = 1; E[5][3] = 1
E[6][2] = 1

#flows = [[x[0] + 0.5*t, x[1] - 0.5*t],
#         [x[0] - 0.5*t, x[1] + 0.8*t],
#         [x[0] + 1.4*t, x[1] - 1.5*t],
#         [x[0] + 3*t,   x[1] - 1.5*t],
#         [x[0] + 2*t,   x[1] - 0.2*t],
#         [x[0] - 1*t,   x[1] + 1.5*t],
#         [x[0] - 0.1*t, x[1] + 0.5*t]]
flows = [[ 0.5, -0.5],
         [-0.5,  0.8],
         [ 1.4, -1.5],
         [ 3  , -1.5],
         [ 2  , -0.2],
         [-1  ,  1.5],
         [-0.1,  0.5]]
# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4)
phi1 = And(x[0] - x[1] <= 10, x[0] - x[1] >= -10, x[0] >= 10, x[0] <= 95, x[1] >= 10, x[1] <= 95) 
phi2 = And(x[0] >= 50, x[0] <= 80, x[1] >= 50, x[1] <= 80, t>= 30, t <= 40)

#phi1 = And(x[0] >= 10, x[0] <= 90, x[1] >= 10, x[1] <= 90) 
#phi2 = And(x[0] >= 60, x[0] <= 80, x[1] >= 40, x[1] <= 70, t>= 50, t <= 60)

given_Init = [And(30 <= x[0], x[0] <= 70, 30 <= x[1], x[1] <= 70, x[1] - x[0] <= 5, x[1] - x[0] >= -5), BoolVal(False), BoolVal(False), BoolVal(False), And(30 <= x[0], x[0] <= 70, 30 <= x[1], x[1] <= 70, x[1] - x[0] <= 5, x[1] - x[0] >= -5), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)

print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")



======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[4][4] finished
X[5][4] finished
X[6][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[4][5] finished
X[5][5] finished
X[6][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[4][6] finished
X[5][6] finished
X[6][6] finished
X[0][7] finished
X[1][7] finished
X[2][7] finished
X[3][7] finished
X[4][7] finished
X[5][7] finished
X[6][7] finished
======================Calculating Time-Irrelevant Jump Conditions================

##### $\varphi_3$

In [20]:

n = 2 # number of variables
m = 7 # number of modes
k = 8 # maximum iteration times

t = Real('t') # time
x = [ Real('x%s' % i) for i in range(n) ] # variables

E = [[0 for i in range(m)] for j in range(m)]

E[0][1] = 1; E[0][5] = 1
E[1][2] = 1; E[1][4] = 1
E[2][1] = 1; E[2][6] = 1
E[3][4] = 1; E[3][5] = 1
E[4][1] = 1
E[5][0] = 1; E[5][3] = 1
E[6][2] = 1

#flows = [[x[0] + 0.5*t, x[1] - 0.5*t],
#         [x[0] - 0.5*t, x[1] + 0.8*t],
#         [x[0] + 1.4*t, x[1] - 1.5*t],
#         [x[0] + 3*t,   x[1] - 1.5*t],
#         [x[0] + 2*t,   x[1] - 0.2*t],
#         [x[0] - 1*t,   x[1] + 1.5*t],
#         [x[0] - 0.1*t, x[1] + 0.5*t]]
flows = [[ 0.5, -0.5],
         [-0.5,  0.8],
         [ 1.4, -1.5],
         [ 3  , -1.5],
         [ 2  , -0.2],
         [-1  ,  1.5],
         [-0.1,  0.5]]
# \varphi = (0 <= x0 <= 4) U[3,4] (3 <= x0 <= 4)
phi1 = And(x[0] >= 10, x[0] <= 95, x[1] >= 10, x[1] <= 95) 
phi2 = And(x[0] >= 50, x[0] <= 80, x[1] >= 50, x[1] <= 80, t>= 30, t <= 40)

#phi1 = And(x[0] >= 10, x[0] <= 90, x[1] >= 10, x[1] <= 90) 
#phi2 = And(x[0] >= 60, x[0] <= 80, x[1] >= 40, x[1] <= 70, t>= 50, t <= 60)

given_Init = [And(30 <= x[0], x[0] <= 70, 30 <= x[1], x[1] <= 70, x[1] - x[0] <= 5, x[1] - x[0] >= -5), BoolVal(False), BoolVal(False), BoolVal(False), And(30 <= x[0], x[0] <= 70, 30 <= x[1], x[1] <= 70, x[1] - x[0] <= 5, x[1] - x[0] >= -5), BoolVal(False), BoolVal(False)]

#q_flow = dict(zip(x,flows[0]))
#X = StateCal(q_flow, phi1, phi2)
#print(X)

print("======================Calculating Time-Relevant Jump Conditions======================")
start1 = time.perf_counter()
X, Init, Jump = TimeRelevantCal(x,E,flows,phi1,phi2,k)
end1 = time.perf_counter()

print("======================Calculating Time-Irrelevant Jump Conditions======================")
start2 = time.perf_counter()
SwitchSyn(x, E, flows, given_Init, Jump, Init, k)
end2 = time.perf_counter()

print(f"                     Alg1                 Alg2      ")
print(f"Total Time  : {end1-start1}s     {end2-start1}s")

prt = 0
if(prt):
     for i in range(k):
          print(f"Init[0][{i}]={Init[0][i]}")
          print(f"Init[1][{i}]={Init[1][i]}")
          print(f"Init[2][{i}]={Init[2][i]}")
          print(f"Init[3][{i}]={Init[3][i]}")
     print("================================================================")
     for q in range(4):
          for post_q in range(4):
               for i in range(k):
                    print(f"Jump[{q}][{post_q}][{i}]={Jump[q][post_q][i]}")
               print("================================================================")

chk = 1
if(chk):
     for i in range(k-1):
          for q in range(m):
               s = Solver()
               s.add(Or(And(Not(X[q][i+1]), X[q][i]), And(X[q][i+1], Not(X[q][i]))))
               if (s.check() == sat):
                    s.add(And(Not(X[q][i+1]), X[q][i]))
                    if(s.check() == sat):
                         print(f"X[{q}][{i}] is not subset of X[{q}][{i+1}]")
                    else:
                         print(f"X[{q}][{i}] is subset of X[{q}][{i+1}], result correct")
               else:
                    print(f"X[{q}][{i}] = X[{q}][{i+1}]")
          print("=====================================================================")



======================Calculating Time-Relevant Jump Conditions======================
X[0][1] finished
X[1][1] finished
X[2][1] finished
X[3][1] finished
X[4][1] finished
X[5][1] finished
X[6][1] finished
X[0][2] finished
X[1][2] finished
X[2][2] finished
X[3][2] finished
X[4][2] finished
X[5][2] finished
X[6][2] finished
X[0][3] finished
X[1][3] finished
X[2][3] finished
X[3][3] finished
X[4][3] finished
X[5][3] finished
X[6][3] finished
X[0][4] finished
X[1][4] finished
X[2][4] finished
X[3][4] finished
X[4][4] finished
X[5][4] finished
X[6][4] finished
X[0][5] finished
X[1][5] finished
X[2][5] finished
X[3][5] finished
X[4][5] finished
X[5][5] finished
X[6][5] finished
X[0][6] finished
X[1][6] finished
X[2][6] finished
X[3][6] finished
X[4][6] finished
X[5][6] finished
X[6][6] finished
X[0][7] finished
X[1][7] finished
X[2][7] finished
X[3][7] finished
X[4][7] finished
X[5][7] finished
X[6][7] finished
======================Calculating Time-Irrelevant Jump Conditions================